In [2]:
# Imports
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from surprise import Dataset
from surprise import Reader
from surprise import SlopeOne

from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from helpersSlopeOne import*

# Load dataset and samples into a pandas data frame
DATA_TRAIN_PATH = 'data/data_train.csv'
data_np = load_data(DATA_TRAIN_PATH)


DATA_TEST_PATH = 'data/sampleSubmission.csv'
samples = load_data(DATA_TEST_PATH)

In [6]:
train_data_np, test_data_np = train_test_split(data_np[['user_id', 'movie_id', 'rating']], test_size = 0.2 ,random_state=1)
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(train_data_np[['user_id', 'movie_id', 'rating']], reader=reader)

In [9]:
trainset = train_data.build_full_trainset()
df = test_data_np[['user_id','movie_id']].copy()
slopeone = SlopeOne().fit(trainset)
    
df['slopeone_rating'] = test_data_np[['user_id', 'movie_id']] \
.apply(lambda row: slopeone.predict(row['user_id'], row['movie_id'])[3], axis=1)

In [10]:
df.to_csv('slopeone_test.csv')

In [7]:
trainset = train_data.build_full_trainset()
df = samples[['user_id','movie_id']].copy()
slopeone = SlopeOne().fit(trainset)
    
df['slopeone_rating'] = samples[['user_id', 'movie_id']] \
.apply(lambda row: slopeone.predict(row['user_id'], row['movie_id'])[3], axis=1)
df.to_csv('slopeone_submission.csv')